In [4]:
import pandas as pd
import numpy as np
import re

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

rm_quoted = re.compile(r"\".*?\"")
arrStopwords = stopwords.words("english")
arrStopwords.extend(["br", "<br>", "<br />"])

## Подготовка данных

In [14]:
data = pd.read_csv("./IMDB Dataset.csv")

In [6]:
countvec = CountVectorizer(stop_words=arrStopwords)

In [7]:
data.iloc[4]["review"]

'Petter Mattei\'s "Love in the Time of Money" is a visually stunning film to watch. Mr. Mattei offers us a vivid portrait about human relations. This is a movie that seems to be telling us what money, power and success do to people in the different situations we encounter. <br /><br />This being a variation on the Arthur Schnitzler\'s play about the same theme, the director transfers the action to the present time New York where all these different characters meet and connect. Each one is connected in one way, or another to the next person, but no one seems to know the previous point of contact. Stylishly, the film has a sophisticated luxurious look. We are taken to see how these people live and the world they live in their own habitat.<br /><br />The only thing one gets out of all these souls in the picture is the different stages of loneliness each one inhabits. A big city is not exactly the best place in which human relations find sincere fulfillment, as one discerns is the case wit

In [8]:
mdata = data

for i in range(len(data)):
    mdata.iloc[i]["review"] = re.sub(rm_quoted, "", mdata.iloc[i]["review"])

vectorized_data = countvec.fit_transform([review for review in mdata["review"]])

for i in range(len(data)):
        mdata.iloc[i]["review"] = vectorized_data[i]                    # .todense() для конвертации в плотную матрицу (и паники ядра, лол)

mdata.replace(to_replace={"sentiment": {"negative": 0, "positive": 1}}, inplace=True)

mdata.head()

,review,sentiment
0,"(0, 62294)\t1\n (0, 73360)\t1\n (0, 55939)...",1
1,"(0, 62294)\t1\n (0, 95490)\t1\n (0, 76515)...",1
2,"(0, 62294)\t1\n (0, 95490)\t1\n (0, 73753)...",1
3,"(0, 32619)\t1\n (0, 95483)\t1\n (0, 48252)...",0
4,"(0, 62294)\t6\n (0, 30123)\t1\n (0, 87877)...",1


In [11]:
type(data.iloc[4]["review"])

scipy.sparse._csr.csr_matrix

## Вычисление модель k-ближайших соседей

In [15]:
test_data, train_data = train_test_split(data, test_size=0.1)
knn = KNeighborsClassifier(n_neighbors=10)
x_train = train_data["review"]
y_train = train_data["sentiment"]

x_test = test_data["review"]
y_test = test_data["sentiment"]

knn_model = knn.fit(
    X=x_train,
    y=y_train
)

knn_predictions = knn.predict(x_test)
accuracy = accuracy_score(y_test, knn_predictions)
display(f"Accuracy score: {accuracy}")

ValueError: could not convert string to float: "At a risk of sounding slightly sacrilegious, on first viewing I'm kind of inclined to put this right up on a par with 'Shaun of the Dead'. Now, given I view Simon Pegg as an unquestionable comedy genius, I realise this is a rather big claim. And to what extent you agree with that last statement may be a good preliminary gauge of whether 'Fido' will appeal to you.<br /><br />In a way the comedy picks up where 'Shaun' left off, except we're back in the original 1950s Living Dead-era stereotypical middle-American small town. The Zombie Wars are over and zombies themselves are becoming more well-adjusted, useful members of the community. This, so we're informed at the outset, is largely thanks to the scientific advances made by the good people at Zomcom - a nice play on romantic comedy perhaps? <br /><br />The beauty of the film lies in its dead-pan depiction of a respectable neighbourhood maintaining core values while making a place for zomb